In [1]:
from datasets import load_dataset

# Cargar el dataset completo y filtrar solo instancias vulnerables (target=1)
dataset_full = load_dataset("bstee615/diversevul", split="train+validation+test")
dataset_full = dataset_full.filter(lambda example: example["target"] == 1)

# Dividir el conjunto en 80% entrenamiento y 20% prueba
ds_split = dataset_full.train_test_split(test_size=0.2, seed=42)
train_ds = ds_split["train"]
test_ds = ds_split["test"]

print(f"Ejemplos de entrenamiento: {len(train_ds)}, Ejemplos de prueba: {len(test_ds)}")
# Ejemplos de entrenamiento: ~15156, Ejemplos de prueba: ~3789 (aprox, dependiendo del dataset exacto)


c:\Users\Berserk\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Filter: 100%|██████████| 330492/330492 [00:02<00:00, 145996.73 examples/s]

Ejemplos de entrenamiento: 15156, Ejemplos de prueba: 3789


In [ ]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Cargar el modelo T5-base pre-entrenado
model = T5ForConditionalGeneration.from_pretrained("t5-base")

# Definir función para calcular métricas de evaluación
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Si se usan generaciones, 'predictions' serán las secuencias generadas (IDs)
    # Reemplazar -100 en labels por el id de <pad> para poder decodificar
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decodificar predicciones y etiquetas a texto
    pred_str  = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Calcular métricas
    acc = accuracy_score(label_str, pred_str)
    prec, rec, f1, _ = precision_recall_fscore_support(label_str, pred_str, average="macro")
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

# Configurar hiperparámetros de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    predict_with_generate=True
)

# Preparar el entrenador con el modelo, datos y métricas
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds_tok,
    eval_dataset=test_ds_tok,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Iniciar el proceso de entrenamiento
trainer.train()


In [8]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Cargar el modelo T5-base pre-entrenado
model = T5ForConditionalGeneration.from_pretrained("t5-base")

# Definir función para calcular métricas de evaluación
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Si se usan generaciones, 'predictions' serán las secuencias generadas (IDs)
    # Reemplazar -100 en labels por el id de <pad> para poder decodificar
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decodificar predicciones y etiquetas a texto
    pred_str  = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Calcular métricas
    acc = accuracy_score(label_str, pred_str)
    prec, rec, f1, _ = precision_recall_fscore_support(label_str, pred_str, average="macro")
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

# Configurar hiperparámetros de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    predict_with_generate=True
)

# Preparar el entrenador con el modelo, datos y métricas
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds_tok,
    eval_dataset=test_ds_tok,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Iniciar el proceso de entrenamiento
trainer.train()


c:\Users\Berserk\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 